<a href="https://colab.research.google.com/github/uwaselari/Data-troubleshooting/blob/main/Copy_of_refs_copies_immutable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part B: References, copies, built-in collections, and immutability #

_(September 1, 2022)_

_Main topics covered in this note:_

- Variables (names), references, and copies
- Primitive types
- Basic collections, e.g., `list`, `set`, `dict`, `tuple`
- Immutability (and hashability)

## References ##

Variables are _names_ for objects. When the objects are "complex" (not "primitive"), modifications through one name may be visible to others.

To wit:

In [ ]:
x = [1, 2, 3, 4, 5]
print("x:", x)

y = x
print("y:", y)

y[2] *= -1
print("Modified y:", y)

**Question:** What is `x`?

In [ ]:
print(x) # What does this produce?

In [ ]:
%%html
<iframe width="800" height="250" frameborder="0" src="https://pythontutor.com/iframe-embed.html#code=x%20%3D%20%5B1,%202,%203,%204,%205%5D%0Ay%20%3D%20x%0Ay%5B2%5D%20*%3D%20-1&codeDivHeight=400&codeDivWidth=350&cumulative=false&curInstr=0&heapPrimitives=nevernest&origin=opt-frontend.js&py=3&rawInputLstJSON=%5B%5D&textReferences=false"> </iframe>

**What's your alternative?** If you really do need a copy, what are your options?

```python
y = [1, 2, 3, 4, 5]
y = x.copy()
y = [e for e in x]
```

Try these in the Python Tutor.

**A tricky case.**

In [ ]:
x = [1, 2, ['a', 'b', 'c'], 4, 5]
y = x.copy()
print(y)

In [ ]:
y[2].append('w')
print(y)

In [ ]:
print(x) # What is the result?

In Python, all unique objects have an _identifier_ associated with them. You can query these.

In [ ]:
id(x), id(y)

In [ ]:
id(x[2]), id(y[2])

In this case, `x` and `y` are distinct objects, but `x[2]` and `y[2]` refer to the same object. When we "copied" `x[2]` into `y[2]`, we copied the `id(x[2])` rather than duplicating the entire object. This kind of copy is sometimes called a _shallow copy_.

Still not clear? Check out a Python Tutor version.

In [ ]:
%%html

<iframe width="1024" height="350" frameborder="0" src="https://pythontutor.com/iframe-embed.html#code=x%20%3D%20%5B1,%202,%20%5B'a',%20'b',%20'c'%5D,%204,%205%5D%0Ay%20%3D%20x.copy%28%29%0Ay%5B2%5D.append%28'w'%29&codeDivHeight=400&codeDivWidth=350&cumulative=false&curInstr=0&heapPrimitives=nevernest&origin=opt-frontend.js&py=3&rawInputLstJSON=%5B%5D&textReferences=false"> </iframe>

**What if you really need a copy for a nested data structure?** The preceding example illustrates that `.copy()` performs a _shallow_ copy. But what if you want a non-shallow, or _deep_, copy? There's a module for that!

In [ ]:
from copy import deepcopy

print('x:', x)
z = deepcopy(x)
print('z:', z)

print('=== appending ===')
z[2].append('@')
print('x:', x)
print('z:', z)

**Exercise** (taken from Notebook 1). Let `L` be a list of strings, e.g.,

```python
L = ['abc', 'def', 'ghi']
```

Complete the function, `rev_str_cat_list(L)` so that it reverses the elements in the list and then concatenates these strings into a single string. It should not modify `L`.

For instance, `rev_str_cat_list(L)` on the above list would return,

```python
'ghidefabc'
```

Your friend supplies the following solution. It appears to produce the correct result, but is wrong. Why?

In [ ]:
def rev_str_cat_list(L):
    L.reverse()
    return ''.join(L)

L = ['abc', 'def', 'ghi']
result = rev_str_cat_list(L)
print(repr(result)) # So right, and yet so wrong. Why?

> _Answer:_ This function is considered _incorrect_ because it modifies its input. Try `print(L)` after the call to `rev_str_cat_list(L)` to verify this claim.
>
> In this case, the exercise stipulates that the function should not modify its input. However, you should always _assume_ that convention unless told otherwise. Why? Remember that you are writing code for others. By adhering to the convention that functions do not modify their inputs, it makes it easier for others to reason about the behavior of your code. When we want your function to modify its input, we will tell you to do so.

## Data structure costs ##

One thing you need to be doing right now is trying to understand the different elementary data structures that Python offers and their tradeoffs.

In this part of the course, the ones you should learn are:

**Tuples** store a _fixed-length_, _ordered sequence_ of values. You can quickly look up any element by its _position_ or _index_. It is "immutable" in the sense that you cannot add or remove elements from the sequence.

**Lists** store a _variable-length_, _ordered sequence_ of values. You can quickly look up any element by its _position_ (or _index_) in the sequence. You can also add or remove elements from the sequence.

**Dictionaries** store key-value pairs. You decide what keys go with what values, where the keys must be unique. You can then quickly look up any value by its _key_.

**Sets** store unique values. They are similar to mathematical sets. You can quickly check whether a value is in the set and perform operations like set intersections, set unions, and so on.

To get a better feel the tradeoffs, let's do a series of experiments. For these experiments, we'll start with a list of 100,000 integer values drawn uniformly from the interval [0, 1 billion].

In [ ]:
from random import randint

n = 100_000
L = [randint(0, 1_000_000_000) for _ in range(n)]

Suppose I want to check whether the number `50` is in the list `L`. I can do that using the expression, `50 in L`. Let's see how long it takes.

In [ ]:
%timeit 50 in L

Suppose I store these values instead in a _set_, `S`. How long does lookup take?

In [ ]:
S = set(L)
print(f'{len(L):,} vs. {len(S):,}') # `len(S)` should be close to `len(L)`

In [ ]:
%timeit 50 in S

Lastly, let's use a dictionary. We'll use the integers themselves as keys and associate them with a dummy value, `True`.

In [ ]:
D = {k: True for k in L}
%timeit 50 in D

## Immutability ##

Certain types of objects in Python are _immutable_, meaning they cannot be modified once they are created. Integers, floating-point values, strings, and tuples are immutable.

Example:

In [ ]:
s = 'abc'
print(s[1])

In [ ]:
#s[1] = 'x'   # Uncomment this code and see what happens

In [ ]:
t = (1, 'a', 3)
#t[1] = '@'   # Uncomment: what happens?

Why is mutability important? The _values_ of a set or the _keys_ of a dictionary _must_ be immutable. Immutability is what allows lookups in those data structures to be fast. (Why?)

In [ ]:
D = {'a': 1, 'b': 2, 'c': 3}
print(D)

In [ ]:
D[(1, 'a', 3)] = 3.14159
print(D)

In [ ]:
D[(1, 'a', 3)]

In [ ]:
{1, 'a', 3} # a set

In [ ]:
#D[{1, 'a', 3}] = 2.71828  # Uncomment: what happens?

**Qualifications.** Two statements above need some qualification.

First, when you use a mutable type as a dictionary key, you'll trigger an error that uses the term "unhashable" rather than "mutable." Immutability and hashability are distinct concepts: immutable objects have values that cannot be modified, whereas hashable objects are simply any object that has a procedure associated with it for converting it into an integer. It is possible for an immutable object to be unhashable, and for a hashable object to be mutable. **However,** _almost_ every immutable type is hashable by convention. (Even the Python documentation on [dictionaries](https://docs.python.org/3/library/stdtypes.html#typesmapping) has some confusing language on this point, referring to both hashability and mutability in the same breath.)

Second, a tuple is immutable under a narrow definition of what its "value" is considered to be. For instance, consider the following code. Based on its output, would you say that a tuple is "immutable" under a common-sense definition?

In [ ]:
immutable_in_air_quotes = (1, ['a', 'b', 'c'], 3)
immutable_in_air_quotes

In [ ]:
immutable_in_air_quotes[1].append('@')
immutable_in_air_quotes

The "narrow definition" of its value is that `id` applied to each component is always the same once you create it. In particular, you cannot change what object exists in each component:

In [ ]:
print(list(id(e) for e in immutable_in_air_quotes))

In [ ]:
#immutable_in_air_quotes[1] = ['different', 'list', 'no', 'way']  # Uncomment: Will error-out

So, since component 1 of `immutable_in_air_quotes` is a list, which is mutable, you can modify it. But the list that you put there can never be replaced by another list.

## Summary ##

1. Every distinct object in Python has an ID, which you can see by `id(x)` for the object `x`.

2. An assignment _copies_ these IDs. That is, in the assignment `y = x`, it will be the case that `id(y)` equals `id(x)`.

3. The built-in primitive types, which are `bool`, `int`, `float`, and `str`, are immutable and effectively copied on assignment. That's because the `id` of any two values is equal if the values are also equal.

In [ ]:
x = 5
y = 5
id(x), id(y)

In [ ]:
x = [1, 2, 3]
y = [1, 2, 3]
id(x), id(y)

4. Shallow vs. deep copies: An object's `.copy()` function will perform a shallow copy. For deep copies, use `deepcopy` from the `copy` module.

5. Python provides several built-in _collections_ or _containers_ for holding a bunch of objects. These are `list`, `set`, `dict`, and `tuple`. You should learn the distinction and start building some intuition for when you might use one or another.

6. For lookups, the `set` and `dict` (for keys) collections can do that quickly. They do so by restricting what you can store in them, namely, _immutable_ objects.